# Model descriptions

The Mitochondrial Retrograde (RTG) Signalling Model is a system of 11 coupled ODEs.

In [ ]:
using RetroSignalModel
using ModelingToolkit

@named sys = RtgMTK(; simplify=false)

## Parameter estimation

Simulated annealing (SAMIN) with bounds was used to find a set of parameters that fit exprerimental conditions. This code example below only runs for 1000 iterations. In our study, we found the solution converges after around 10^5 iterations. Moreover, we can use another function `optim_params_threads()` to generate valid parameter sets in paralell.

[Source code](https://github.com/ntumitolab/RetroSignalModel.jl/blob/main/src/params.jl)

In [ ]:
using RetroSignalModel
import RetroSignalModel as rs
using Optim

res, params = optim_params(
    rs.RtgMTK;
    knockoutlevel=1e-4,
    proteinlevels=rs.STRESSED,
    lowerbound=1e-3,
    upperbound=1e3,
    hilllowerbound=2.0,
    hillupperbound=5.0,
    xinit=1.0,
    optimsolver=Optim.SAMIN(),
    optimoptions=Optim.Options(iterations=10^3, show_trace=true, show_every=100),
    targetratio=10)

## Valid parameter sets

In [6]:
using RetroSignalModel
using CSV
using DataFrames
using Plots

In [44]:
filename = joinpath(dirname(pathof(RetroSignalModel)), "data", "solution_rtgMTK_optim.csv")
dfoptim = let 
    df = CSV.read(filename, DataFrame)
    df[!, Not(:n_S)] .= log10.(df[!, Not(:n_S)])
end

res = map(sort(names(dfoptim))) do lab
    histogram(dfoptim[!, lab], label=lab)
end

plot(res..., layout=(6, 4), size=(1024, 1024))
# savefig("optimparams.png")

In [45]:
filename = joinpath(dirname(pathof(RetroSignalModel)), "data", "solution_rtgM4.csv")
dforig = let 
    df = CSV.read(filename, DataFrame)
    df[!, Not(:n_S)] .= log10.(df[!, Not(:n_S)])
end

res = map(sort(names(dforig))) do lab
    histogram(dforig[!, lab], label=lab)
end

plot(res..., layout=(6, 4), size=(1024, 1024))
# savefig("randomparams.png")